# <h1><center><font color='green'>**IoT data manipulation and visualisation for olive trees monitoring**</font></center></h1>
<div class="container" style="display: flex; align-items: center; justify-content: center">
    <div class="image">
        <img src="./images/IoT_image.png" style="float: left;" width="800"/>
    </div>
    <div class="text" style="padding-left: 50px;">
        <p> This notebook aims to mainipulate and visualise data coming from TreeTalker sensors installed on trees in two olive fields located in Mesagne and Avetrana in Apulia region.
        </p>
        <p>
            Pandas is a powerful and easy to use Python library used for data manipulation.
        </p>
    </div>
</div>

<hr style="height:1px;border-top:1px solid #0000FF" />

In the following cell, the `Pandas` and `System` libraries required to access and process data are imported into the notebook. </br>
Then, the path to the folder `MAGICASummerSchool` is specified.

In [ ]:
import pandas as pd
import sys
sys.path.insert(0,'.')

`support_libraries` contains ad-hoc functions for data visualisation: </br>
*   `getMap` creates a map visualising the sensors position on the field
*   `getTable` shows the data on a table
*   `getGraph` visualises the time series of a chosen variable

In [ ]:
%%capture
from support_libraries import getMap, getTable, reduce, getGraph

<hr style="height:1px;border-top:1px solid #0000FF" />

### Visualise TreeTalker Map

Here you can choose between the two fields (Avetrana and Mesagne) monitored by the TreeTalker sensors in Apulia region. A map will appear, after clicking on the "View map" button, showing the position of the sensors on the field. </br>

In [ ]:
getMap()

<hr style="height:1px;border-top:1px solid #0000FF" />

### Visualise TreeTalker data in a Table

The `getTable` function gets the site name as input ('Avetrana' or 'Mesagne') and visualises a table containing all the variables values for each timestamp and TreeTalker serial number.</br>
In the following, a detailed description of each column:

*   `timestamp` --> Time of measurement detection by the sensor;
*   `tt_serial_number` --> sensor serial number, which is a numerical code that corresponds to the TreeTalker identifier;
*   `air_temperature` --> air temperature near the sensor;
*   `air_humidity` --> air humidity near the sensor;
*   `sapflow` --> an indicator of fluid movement in tree stems;
*   `growth_rate` --> measurement of the increase in tree trunk diameter;
*   `VPD` *(Vapour Pressure Deficit)* --> an indicator of air dryness and likely subsequent leaf dehydration;
*   `NDVI` *(Normalized Difference Vegetation Index)* --> a biomass-related indicator used to assess the level of vigor and vitality of the tree;
*   `NPP` *(Net Primary Productivity)* --> an indicator obtained as the difference between the carbon from photosynthesis and the carbon released from tree respiration.

In [ ]:
data = getTable('Avetrana')
data

<hr style="height:1px;border-top:1px solid #0000FF" />

### Data Filtering

It is possible to filter data based on certain parameters. Some examples are given below:
* `data[data["tt_serial_number"] == '52070729']` --> filters the rows of the table by returning only the measurements taken by the sensor with the specified serial number;
* `data[data["air_humidity"] > 70]` --> filters the rows of the table by taking only those that have an air humidity value greater than 70%;
* `subset = data[data["NDVI"] < 0]` --> filters the rows in the table that have a negative NDVI value;
* `data[data["air_temperature"].between(20, 40)]` --> filters the rows of the table and returns only those that have a temperature value between 20°C and 40°C;
* `data[data.index == '2021-03-10 01:00:00'][['tt_serial_number','growth_rate']]` --> filters the rows and columns returning only those sensors and growth rates that have a specified timestamp;
* `data[data.index.year == 2021]` --> returns only values for the year 2021.

In [ ]:
subset = data[data['tt_serial_number'] == '52070729']
# subset = data[data['air_humidity'] > 70]
# subset = data[data['NDVI'] < 0]
# subset = data[data['air_temperature'].between(20, 40)]
# subset = data[data.index == '2021-03-10 01:00:00'][['tt_serial_number','growth_rate']]
# subset = data[data.index.year == 2021]
subset

<hr style="height:1px;border-top:1px solid #0000FF" />

### Data Plotting

`matplotlib` is the most used graphical plotting library.
- Pandas implicitly uses `matplotlib.pyplot` to plot the data of a dataframe.
- This allows to make plots in a very simple and immediate way.

In [ ]:
data[(data.index.year == 2021) & (data.index.month == 10)]['air_temperature'].plot()

Many plot styles are available in `matplotlib`.

In [ ]:
import matplotlib.pyplot as plt
import datetime

plt.style.use('ggplot')
data[data['tt_serial_number'] == '52070729']
subset = data[(data['tt_serial_number'] == '52070729')&(data.index.date == datetime.date(2021, 3, 10))]['NDVI']
subset.plot(kind='bar')
plt.ylabel('NDVI')

You can also graph many datasets together.

In [ ]:
# Select data coming from two different sensors.
subset_vpd = data[data['tt_serial_number'] == '52070725'].loc['2021-03-10 10:00:00':'2021-03-10 17:00:00']['VPD']
subset_ndvi = data[data['tt_serial_number'] == '52070729'].loc['2021-03-10 10:00:00':'2021-03-10 17:00:00']['VPD']

# Trace the graph using different colored markers.
plt.plot(subset_vpd, 'b-', label='VPD_52070725')
plt.plot(subset_ndvi, 'g-', label='VPD_52070729')

# Create the legend.
plt.legend(loc='upper right')
plt.ylabel('VPD')

<hr style="height:1px;border-top:1px solid #0000FF" />

### TIME SUBSETTING

1. Below is an example of `time subsetting` using an instruction that selects a treetalker and displays the time graph by choosing a variable and a time interval.

In [ ]:
subset = data[data['tt_serial_number'] == '52070729'].loc['2021-03-10 01:00:00':'2021-08-10 01:00:00']['air_temperature']
# subset = data[(data['tt_serial_number'] == '52070729') & (data.index.year == 2022)]['air_temperature']
subset

In [ ]:
getGraph(subset, metric='air_temperature', frequency='hourly')

<hr style="height:1px;border-top:1px solid #0000FF" />

### TIME AGGREGATION

2. Below is an example of `time aggregation` by selecting a treetalker and displaying the table and graph of the daily average, including highest and lowest temperature.

In [ ]:
subset2 = data[data['tt_serial_number'] == '52070733']
reduced_data2 = reduce(subset2, metric='air_temperature', frequency='daily')
reduced_data2

In [ ]:
getGraph(reduced_data2, metric='air_temperature', frequency='daily')

<hr style="height:1px;border-top:1px solid #0000FF" />

### SPACE AND TIME AGGREGATION

3. This is an example of `space and time aggregation`. The table and graph of the daily average temperature in the field is displayed.

In [ ]:
reduced_data3 = reduce(data, metric='air_temperature', frequency='daily', operation='avg')
reduced_data3

In [ ]:
getGraph(reduced_data3, metric='air_temperature', frequency='daily')

3b. Another example of `time and space aggregation` by displaying the weekly maximum temperature in the field.

In [ ]:
reduced_data3b = reduce(data, metric='air_temperature', frequency='weekly', operation='max')
reduced_data3b

In [ ]:
getGraph(reduced_data3b, metric='air_temperature', frequency='weekly')

<hr style="height:1px;border-top:1px solid #0000FF" />

4. Here is a further example of displaying the average, highest and lowest weekly VPD (Vapour Pressure Deficit) in the field. Again, this is `space and time aggregation`.

In [ ]:
reduced_data4 = reduce(data, metric='VPD', frequency='weekly')
reduced_data4

In [ ]:
getGraph(reduced_data4, metric='VPD', frequency='weekly')

<hr style="height:1px;border-top:1px solid #0000FF" />

5. A final example of displaying the average, highest and lowest monthly air humidity in the field.

In [ ]:
reduced_data5 = reduce(data, metric='air_humidity', frequency='monthly')
reduced_data5

In [ ]:
getGraph(reduced_data5, metric='air_humidity', frequency='monthly')

<hr style="height:1px;border-top:1px solid #0000FF" />

Next: [**TreeTalker Exercises**](TreeTalkerExercises.ipynb)